In [4]:
import numpy as np
x=np.load('../../../datasets/COIL100/d_coil100.npy')
x=x.reshape(7200,3000)
x_test=x[5760:,:]
y=np.load('../../../datasets/COIL100/l_coil100.npy')
y=y.reshape(7200,)
y_test=y[5760:,]
y_test=y_test.reshape(1440,)

x_train=np.load('../../../datasets/COIL100/d_coil100train.npy')
x_train=x_train.reshape(5760,3000)
y=np.load('../../../datasets/COIL100/l_coil100train.npy')
y=y.reshape(5760,)

alpha_trans1=np.load('../../../datasets/COIL100/Sb_coil100train.npy',allow_pickle=True)
alpha_trans1=np.asarray(alpha_trans1).astype('float32') 

In [5]:

import glob
import numpy as np
import random
import tensorflow.compat.v1 as tf
import os
from sklearn.utils import shuffle
batchSize=720
from sklearn.metrics.pairwise import euclidean_distances
from scipy.linalg import fractional_matrix_power
from scipy import linalg
import sklearn
from sklearn.cluster import KMeans
from tensorflow.keras import backend as B
k_cluster = 100
from tensorflow.keras.layers import Input,Dense,Lambda,Layer
from tensorflow.keras.models import Model
import sys
import numpy as np
from scipy.io import loadmat
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix
from munkres import Munkres

from numpy import save


def eig_vec(x_train):
    nev_trans=[]
    SortedEigVec=np.zeros((batchSize,batchSize))
    for l in range (0,len(x_train),batchSize):
        xtr=x_train[l:l+batchSize]
        d=(euclidean_distances(xtr,xtr))
        W=np.zeros((len(xtr),len(xtr)))
        for i in range(len(xtr)):
            for j in range (len(xtr)):
                if i==j:
                    W[i,j]=0
                else:
                    W[i,j]=np.exp(-(d[i,j]**2)/(2*0.09))
        degree=np.diag(sum(W))
        L = degree-W
        eig_val,eig_vec=np.linalg.eig(L)
        dec_eigVal=np.flip(np.sort(eig_val))
        indexArr=(np.flip(np.argsort(eig_val)))
        for i in range (len(indexArr)):
            SortedEigVec[:,i]=eig_vec[:,indexArr[i]]
        SortedEigVec=np.array(SortedEigVec)
        nev_trans.append(SortedEigVec)
    return nev_trans,L,eig_val,eig_vec,dec_eigVal,W


def make_cost_matrix(c1, c2):
    uc1 = np.unique(c1)
    uc2 = np.unique(c2)
    l1 = uc1.size
    l2 = uc2.size
    for a in range (1):
        if uc1.shape==uc2.shape:
            m = np.ones([l1, l2])
            for i in range(l1):
                it_i = np.nonzero(c1 == uc1[i])[0]
                for j in range(l2):
                    it_j = np.nonzero(c2 == uc2[j])[0]
                    m_ij = np.intersect1d(it_j, it_i)
                    m[i,j] =  -m_ij.size
        else:
            print('assertion handeled')
            return
            #break
    return m

def translate_clustering(clt, mapper):
    return np.array([ mapper[i] for i in clt ])

def accuracy(cm):
    return np.trace(cm, dtype=float) / np.sum(cm)

def rectify_label(labels, classes):
    num_labels = len(np.unique(classes))
    cm = confusion_matrix(classes, labels, labels=range(num_labels)) # gets the confusion matrix
    cost_matrix = make_cost_matrix(labels, classes)
    if cost_matrix is None:
        return (None,None)
    else:
        m = Munkres()
        indexes = m.compute(cost_matrix)
        mapper = { old: new for (old, new) in indexes }
        new_labels = translate_clustering(labels, mapper)
        new_cm = confusion_matrix(classes, new_labels, labels=range(num_labels))
        return new_labels,accuracy(new_cm)

def arch():
    encoding_dim = 710  
    input_img = Input(shape=(3000,))
    encoded_layer_2 = Dense(2000, activation='tanh')(input_img)
    encoded_layer_3 = Dense(1300, activation='tanh')(encoded_layer_2)
    encoded_layer_4 = Dense(800, activation='tanh')(encoded_layer_3)
    encoded = Dense(encoding_dim, activation='tanh')(encoded_layer_4)
    decoded_layer_2 = Dense(800, activation='tanh')(encoded)
    decoded_layer_3 = Dense(1300, activation='tanh')(decoded_layer_2)
    decoded_layer_4 = Dense(2000, activation='tanh')(decoded_layer_3)
    decoded = Dense(3000, activation='tanh')(decoded_layer_4)
    encoder=Model(input_img,encoded)
    autoencoder=Model(input_img,decoded)
    return encoder,encoding_dim,autoencoder
  
def assign_func(z_test,z_ic):
    dst = np.array(np.sum(((z_test-z_ic[0]) **2),axis=1))
    for index in range(1, z_ic.shape[0]):
        col = np.sum(((z_test- z_ic[index])**2), axis=1)
        dst = np.vstack((dst, col))
    dst = dst.T      
    
    mean_dst = np.mean(dst, axis=1)
    q = np.maximum(0.0, np.tile(mean_dst, (dst.shape[1], 1)).T - dst)
    #print(q.shape)
    num_centers = q.shape[1]
    weight = 1.0 / (q.sum(axis=0) + 1e-9)
    weight *= num_centers / weight.sum()
    q = (q ** 2.0) * weight
    q = (q.T / (q.sum(axis=1) + B.epsilon())).T
    return q

def asl():
    def myloss(y_true, y_pred):
        l1norm= l1norm_func(y_pred,y_true)
        l2norm= l2norm_func(y_pred,y_true)
        orthonormal= orthonormality_func(y_pred)
        structpreserve=structurePreserve_func(y_pred,alpha_trans1[g])
        return l2norm+structpreserve+orthonormal+l1norm
    return myloss

def l1norm_func(z,nt):
    q1=0.002*(tf.reduce_sum(tf.abs(tf.subtract(z,nt)),axis=1))
    return q1

def l2norm_func(z,nt):
    q2=0.02*(tf.sqrt(tf.reduce_sum(tf.square(z-nt))))
    return q2


def structurePreserve_func(z,ta):
    tz=tf.transpose(z)
    tz=tf.reshape(tz,(encoding_dim,batchSize))
    taa=tf.transpose(ta)
    taz=tf.matmul(tz,taa)
    taz=tf.reshape(taz,(encoding_dim,batchSize))
    q4=0.02*(tf.sqrt(tf.reduce_sum(tf.square(tz-taz))))
    q4=tf.transpose(q4)
    return q4

def orthonormality_func(z):
    z=[(z[i,:]/(tf.norm(z[i,:],2)))for i in range (batchSize)]
    transz=tf.transpose(z)
    q3=tf.matmul(transz,z)
    I=tf.eye(encoding_dim)
    q5=0.002*(tf.sqrt(tf.reduce_sum(tf.square(q3-I))))
    return q5



In [6]:
#nev_trans,L,eig_val,eig_vec,dec_eigVal,W=eig_vec(x_train)

In [4]:
# encoder,encoding_dim,autoencoder=arch()
# autoencoder.compile(tf.keras.optimizers.Adam(lr=1e-3), loss=tf.keras.losses.MeanSquaredError())
# autoencoder.fit(x_train,x_train,epochs=100,shuffle=True,batch_size=batchSize)
# encoder.save_weights('./datasets/coil100/Coil100train.ckpt')

# encoder.load_weights('./datasets/coil100/Coil100train.ckpt')
# g=0
# encoder.compile(tf.keras.optimizers.Adadelta(lr=1e-3, rho=0.95, decay=1e-4), loss=asl())

# maxnum=0
# for t in range (1,900,1):
#     g=0
#     for l in range (0,len(x_train),batchSize):
#         encoder.fit(x_train[l:l+batchSize],nev_trans[g][:,batchSize-encoding_dim-1:batchSize-1],epochs=1,shuffle=False,batch_size=batchSize)
#         g=g+1
#     z_test=encoder.predict(x_train)
#     k_means = KMeans(n_clusters=k_cluster, init='k-means++')
#     ic= k_means.fit(z_test).cluster_centers_ 
#     assignment = assign_func(z_test, ic)
#     predictedlabels = np.argmax(assignment, axis=1)
#     rectified_label, new_acc=rectify_label(predictedlabels, y)
#     if rectified_label is not None:
#         print('acc at',t, 'epoch is:',maxnum)
#         if maxnum<new_acc:
#             maxnum=new_acc
#             encoder.save_weights('../../../checkpoints/chk_coil100/sdsc_Coil100train.ckpt'')
#             np.save('../../../checkpoints/chk_coil100/sdsc_icCoil100train.npy',ic)
#             maxnmi=sklearn.metrics.normalized_mutual_info_score(y, predictedlabels)
#             maxf1=sklearn.metrics.f1_score(y, rectified_label,average='micro')
#             maxprec=sklearn.metrics.precision_score(y, rectified_label,average='weighted')
#     else:
#         continue


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 5760 samples
Epoch 1/100
5760/5760 [==============================] - 2s 319us/sample - loss: 4.9119e-04
Epoch 2/100
5760/5760 [==============================] - 1s 192us/sample - loss: 3.7424e-04
Epoch 3/100
5760/5760 [==============================] - 1s 191us/sample - loss: 3.2288e-04
Epoch 4/100
5760/5760 [==============================] - 1s 193us/sample - loss: 2.8232e-04
Epoch 5/100
5760/5760 [==============================] - 1s 193us/sample - loss: 2.4887e-04
Epoch 6/100
5760/5760 [==============================] - 1s 190us/sample - loss: 2.2346e-04
Epoch 7/100
5760/5760 [==============================] - 1s 192us/sample - loss: 2.0411e-04
Epoch 8/100
5760/5760 [==============================] - 1s 193us/sample - loss: 1.8859e-04
Epoch 9/100
5760/5760 [==============================] - 1s 192us/sample - loss: 1.7560e-04
Epoch 10/100
5760/5760 [==============================] - 1s 195us/sa

Epoch 85/100
5760/5760 [==============================] - 1s 190us/sample - loss: 4.1023e-05
Epoch 86/100
5760/5760 [==============================] - 1s 191us/sample - loss: 4.0741e-05
Epoch 87/100
5760/5760 [==============================] - 1s 190us/sample - loss: 4.0468e-05
Epoch 88/100
5760/5760 [==============================] - 1s 190us/sample - loss: 4.0191e-05
Epoch 89/100
5760/5760 [==============================] - 1s 191us/sample - loss: 3.9932e-05
Epoch 90/100
5760/5760 [==============================] - 1s 195us/sample - loss: 3.9664e-05
Epoch 91/100
5760/5760 [==============================] - 1s 194us/sample - loss: 3.9408e-05
Epoch 92/100
5760/5760 [==============================] - 1s 191us/sample - loss: 3.9161e-05
Epoch 93/100
5760/5760 [==============================] - 1s 192us/sample - loss: 3.8920e-05
Epoch 94/100
5760/5760 [==============================] - 1s 193us/sample - loss: 3.8668e-05
Epoch 95/100
5760/5760 [==============================] - 1s 193us/sam

Train on 720 samples
720/720 [==============================] - 0s 439us/sample - loss: 1.9466
Train on 720 samples
720/720 [==============================] - 0s 433us/sample - loss: 1.9523
Train on 720 samples
720/720 [==============================] - 0s 432us/sample - loss: 1.9454
Train on 720 samples
720/720 [==============================] - 0s 436us/sample - loss: 1.9456
acc at 9 epoch is: 0.8090277777777778
Train on 720 samples
720/720 [==============================] - 0s 436us/sample - loss: 1.8970
Train on 720 samples
720/720 [==============================] - 0s 431us/sample - loss: 1.9439
Train on 720 samples
720/720 [==============================] - 0s 432us/sample - loss: 1.9488
Train on 720 samples
720/720 [==============================] - 0s 432us/sample - loss: 1.9480
Train on 720 samples
720/720 [==============================] - 0s 426us/sample - loss: 1.9437
Train on 720 samples
720/720 [==============================] - 0s 433us/sample - loss: 1.9494
Train on 720

Train on 720 samples
720/720 [==============================] - 0s 463us/sample - loss: 1.9183
acc at 19 epoch is: 0.8090277777777778
Train on 720 samples
720/720 [==============================] - 0s 435us/sample - loss: 1.8698
Train on 720 samples
720/720 [==============================] - 0s 430us/sample - loss: 1.9162
Train on 720 samples
720/720 [==============================] - 0s 432us/sample - loss: 1.9210
Train on 720 samples
720/720 [==============================] - 0s 430us/sample - loss: 1.9206
Train on 720 samples
720/720 [==============================] - 0s 434us/sample - loss: 1.9164
Train on 720 samples
720/720 [==============================] - 0s 434us/sample - loss: 1.9219
Train on 720 samples
720/720 [==============================] - 0s 435us/sample - loss: 1.9151
Train on 720 samples
720/720 [==============================] - 0s 437us/sample - loss: 1.9157
acc at 20 epoch is: 0.8090277777777778
Train on 720 samples
720/720 [==============================] - 0s 

Train on 720 samples
720/720 [==============================] - 0s 444us/sample - loss: 1.8907
Train on 720 samples
720/720 [==============================] - 0s 431us/sample - loss: 1.8954
Train on 720 samples
720/720 [==============================] - 0s 430us/sample - loss: 1.8954
Train on 720 samples
720/720 [==============================] - 0s 435us/sample - loss: 1.8912
Train on 720 samples
720/720 [==============================] - 0s 430us/sample - loss: 1.8964
Train on 720 samples
720/720 [==============================] - 0s 430us/sample - loss: 1.8899
Train on 720 samples
720/720 [==============================] - 0s 441us/sample - loss: 1.8908
acc at 30 epoch is: 0.8090277777777778
Train on 720 samples
720/720 [==============================] - 0s 435us/sample - loss: 1.8422
Train on 720 samples
720/720 [==============================] - 0s 436us/sample - loss: 1.8882
Train on 720 samples
720/720 [==============================] - 0s 431us/sample - loss: 1.8930
Train on 72

...

KeyboardInterrupt: 

Exception ignored in: 'sklearn.cluster._k_means_fast._relocate_empty_clusters_dense'
Traceback (most recent call last):
  File "<__array_function__ internals>", line 2, in where
KeyboardInterrupt


KeyboardInterrupt: 

Exception ignored in: 'sklearn.cluster._k_means_fast._relocate_empty_clusters_dense'
Traceback (most recent call last):
  File "<__array_function__ internals>", line 2, in where
KeyboardInterrupt


KeyboardInterrupt: 

In [9]:
encoder,encoding_dim,autoencoder=arch()
encoder.load_weights('../../../checkpoints/chk_coil100/sdsc_Coil100train.ckpt')
z_test=encoder.predict(x_test)
ic= np.load('../../../checkpoints/chk_coil100/sdsc_icCoil100train.npy')
assignment = assign_func(z_test, ic)
predictedlabels = np.argmax(assignment, axis=1)
rectified_label, acc=rectify_label(predictedlabels, y_test)
nmi=sklearn.metrics.normalized_mutual_info_score(y_test, predictedlabels)
f1=sklearn.metrics.f1_score(y_test,rectified_label,average='micro')
prec=sklearn.metrics.precision_score(y_test, rectified_label,average='weighted')
print(acc,nmi,f1,prec)

0.8604166666666667 0.9417541191665969 0.8604166666666667 0.862689152219894
